<a href="https://colab.research.google.com/github/IrinaBoyarchukova/A_B_test/blob/main/gb_sem_7%D0%9D%D0%B5%D1%81%D1%82%D0%B0%D0%BD%D0%B4%D0%B0%D1%80%D1%82%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%82%D1%83%D0%B0%D1%86%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# импорт необходимых библиотек
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm

import scipy.stats as st
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats import proportion
from statsmodels.stats.meta_analysis import effectsize_smd

### Задание на семинар
1. Мы провели А/А тест, в котором сравнивали одну и ту же версию сайта. Полученные результаты выглядят следующим образом. Можем ли мы на основе проведенного А/А теста запустить А/В-тестирование? Используйте уровень доверия 95%.
|                | **Версия А1** | **Версия А2** |
|:--------------:|:------------:|:------------:|
|   **Выборка**  |     1731     |     1702     |
| **Транзакция** |      152     |      189     |

In [2]:
# Задаем начальные параметры
alpha = .05
prob_1, prob_2 = 152 / 1731, 189 / 1702
sample_1_size, sample_2_size = 1731, 1702
# Генерируем распределение
a = st.bernoulli.rvs(p=prob_1, size=sample_1_size, random_state=12)
b = st.bernoulli.rvs(p=prob_2, size=sample_2_size, random_state=25)

In [3]:
# Сравнение хи2 и z
chi_value, p_value_chi2, table = proportion.proportions_chisquare([a.sum(), b.sum()], [a.size, b.size])
z_value, p_value_z = proportion.proportions_ztest([a.sum(), b.sum()], [a.size, b.size])
print(p_value_chi2, p_value_z)

0.02286339922157709 0.02286339922157713


In [4]:
# Считаем статистику по выборкам
chi_value, p_value, table = proportion.proportions_chisquare([a.sum(), b.sum()], [a.size, b.size])

se = np.sqrt((prob_1 * (1 - prob_1) / a.size) + (prob_2 * (1 - prob_2) / b.size))
md = a.mean() - b.mean()
left = md - 1.96 * se
right = md + 1.96 * se

print(f"Mean difference: {np.round(md, 5)}, p-value: {p_value} and confidence interval is: [{np.round(left, 5)}, {np.round(right, 5)}]")

if p_value < alpha and not min(left, right) < 0 < max(left, right):
    print(f"Sample # {1 if md > 0 else 2} mean is greater")
else:
    print("No significant difference")

Mean difference: -0.02324, p-value: 0.02286339922157709 and confidence interval is: [-0.04325, -0.00322]
Sample # 2 mean is greater


In [5]:
# Альтернативный способ через бутсрап
booted_diff = []
booted_pvalue = []
size = max(a.size, b.size)
for _ in tqdm(range(10_000)):
    a_s = st.bernoulli.rvs(p=prob_1, size=size)
    b_s = st.bernoulli.rvs(p=prob_2, size=size)
    booted_diff.append(np.mean(a_s - b_s))
    booted_pvalue.append(proportion.proportions_chisquare([a_s.sum(), b_s.sum()], [a_s.size, b_s.size])[1])

md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
p_value_ci = 2 * (1 - st.norm.cdf(np.abs(md_ci / std_ci)))

100%|██████████| 10000/10000 [00:07<00:00, 1420.67it/s]


In [6]:
# Доверительный интервал разниц между группами
print(f"Mean difference: {np.round(md_ci, 5)}, p-value: {p_value_ci} and confidence interval is: [{np.round(left_ci, 5)}, {np.round(right_ci, 5)}]")

if p_value_ci < alpha and not min(left_ci, right_ci) < 0 < max(left_ci, right_ci):
    print(f"Sample # {1 if md_ci > 0 else 2} mean is greater")
else:
    print("No significant difference")

Mean difference: -0.02327, p-value: 0.021777101901959695 and confidence interval is: [-0.04275, -0.00347]
Sample # 2 mean is greater


In [7]:
# Ошибка первого рода, которая говорит нам о том, что группы различаются между собой!
np.mean(np.array(booted_pvalue) < 0.05) # FPR

0.6276

In [8]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Histogram(x=booted_pvalue, name='booted_pvalue',),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=booted_diff, name='booted_diff',),
    row=2, col=1
)

fig.update_layout(height=800, width=1000, title_text="After bootstrap")
fig.show()

### Задание на семинар
2. Мы провели А/А тест, в котором сравнивали одну и ту же версию сайта. Полученные результаты выглядят следующим образом. Можем ли мы на основе проведенного А/А теста запустить А/В-тестирование? Используйте уровень доверия 95%.
|                             | **Версия А1** | **Версия А2** |
|:---------------------------:|:-------------:|:-------------:|
|         **Среднее**         |     38.175    |     33.710    |
| **Стандартное  отклонение** |       28      |       11      |
|         **Выборка**         |      108      |      105      |

In [9]:
# Вводим исходные данные по выборкам
alpha = .05
sample_1_mean, sample_2_mean = 38.175, 33.710 
sample_1_se, sample_2_se = 28, 11
sample_1_size, sample_2_size = 108, 105
# Генерируем нормальное распределение на основе введенных данных
norm_1 = st.norm(loc=sample_1_mean, scale=sample_1_se).rvs(size=sample_1_size, random_state=42)
norm_2 = st.norm(loc=sample_2_mean, scale=sample_2_se).rvs(size=sample_2_size, random_state=42)

In [ ]:
# Вводим исходные данные по выборкам
alpha = .05
sample_1_mean, sample_2_mean = 38.175, 38.710 
sample_1_se, sample_2_se = 28, 11
sample_1_size, sample_2_size = 1080, 1050
# Генерируем нормальное распределение на основе введенных данных
norm_1 = st.norm(loc=sample_1_mean, scale=sample_1_se).rvs(size=sample_1_size, random_state=42)
norm_2 = st.norm(loc=sample_2_mean, scale=sample_2_se).rvs(size=sample_2_size, random_state=42)

In [10]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=norm_1, name='norm_1'))
fig.add_trace(go.Histogram(x=norm_2, name='norm_2'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [11]:
# Считаем статистику по выборкам
tvalue, pvalue = st.ttest_ind(norm_1, norm_2, equal_var=False, alternative='two-sided')
md = sample_1_mean - sample_2_mean
se = md / tvalue

left, right = st.norm.interval(.95, loc=md, scale=se)
print(f"Mean difference: {np.round(md, 5)}, p-value: {p_value} and confidence interval is: [{np.round(left, 5)}, {np.round(right, 5)}]")

if p_value < alpha and not min(left, right) < 0 < max(left, right):
    print(f"Sample # {1 if md > 0 else 2} mean is greater")
else:
    print("No significant difference")

Mean difference: 4.465, p-value: 0.02286339922157709 and confidence interval is: [-3.1737, 12.1037]
No significant difference


In [12]:
# Альтернативный способ через бутсрап
booted_diff = []
booted_pvalue = []
size = max(norm_1.size, norm_2.size)
for _ in tqdm(range(10_000)):
    a_s = np.random.choice(norm_1, size=size, replace=True)
    b_s = np.random.choice(norm_2, size=size, replace=True)
    booted_diff.append(np.mean(a_s - b_s))
    booted_pvalue.append(st.ttest_ind(a_s, b_s, equal_var=False, alternative='two-sided')[1])

# Доверительный интервал разниц между группами
md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
p_value_ci = 2 * (1 - st.norm.cdf(np.abs(md_ci / std_ci)))

100%|██████████| 10000/10000 [00:04<00:00, 2326.14it/s]


In [13]:
print(f"Mean difference: {np.round(md_ci, 5)}, p-value: {p_value_ci} and confidence interval is: [{np.round(left_ci, 5)}, {np.round(right_ci, 5)}]")

if p_value_ci < alpha and not min(left_ci, right_ci) < 0 < max(left_ci, right_ci):
    print(f"Sample # {1 if md_ci > 0 else 2} mean is greater")
else:
    print("No significant difference")

Mean difference: 3.03854, p-value: 0.24446555727083963 and confidence interval is: [-2.0328, 8.04182]
No significant difference


In [14]:
# Ошибка первого рода, которая говорит нам о том, что группы почти не различаются между собой!
np.mean(np.array(booted_pvalue) < 0.05) # FPR

0.2124

In [15]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Histogram(x=booted_pvalue, name='booted_pvalue',),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=booted_diff, name='booted_diff',),
    row=2, col=1
)

fig.update_layout(height=800, width=1000, title_text="After bootstrap")
fig.show()

### Задание на семинар
3. Мы запустили A/B-тест. На его проведение мы собираемся потратить 1 неделю, на каждую выборку направить по 10000 юзеров. На третий день, продакт посмотрел на промежуточные результаты. На этот момент в каждой выборке было по 3000 юзеров, а результат в конверсиях был статистически значимым в пользу версии B. Можем ли мы на основе таких данных выпустить версию B на всех юзеров?


### Задание на семинар
4. Мы провели А/А тест, в котором сравнивали одну и ту же версию сайта. Полученные результаты выглядят следующим образом. Можем ли мы на основе проведенного А/А теста запустить А/В-тестирование? Используйте уровень доверия 95%.
|                          | **Версия А1** | **Версия A2** |
|:------------------------:|:------------:|:------------:|
|       **Посещение**      |     6351     |     6442     |
| **Добавление в корзину** |     1754     |     1902     |
| **Подтверждение заказа** |     1025     |     1099     |
|      **Транзакция**      |      623     |      642     |

In [58]:
# Задаем начальные параметры
alpha = .05
sample_1_size, sample_2_size = 6351, 6442
prob_1, prob_2 = 623 / sample_1_size, 642 / sample_2_size
# Генерируем распределение
a = st.bernoulli.rvs(p=prob_1, size=sample_1_size, random_state=12)
b = st.bernoulli.rvs(p=prob_2, size=sample_2_size, random_state=25)

In [64]:
# Задаем начальные параметры
alpha = .05
sample_1_size, sample_2_size = 6351, 6442
prob_1, prob_2 = 1025 / sample_1_size, 1099 / sample_2_size
# Генерируем распределение
a = st.bernoulli.rvs(p=prob_1, size=sample_1_size, random_state=12)
b = st.bernoulli.rvs(p=prob_2, size=sample_2_size, random_state=25)

In [69]:
# Задаем начальные параметры
alpha = .05
sample_1_size, sample_2_size = 6351, 6442
prob_1, prob_2 = 1754 / sample_1_size, 1902 / sample_2_size
# Генерируем распределение
a = st.bernoulli.rvs(p=prob_1, size=sample_1_size, random_state=12)
b = st.bernoulli.rvs(p=prob_2, size=sample_2_size, random_state=25)

In [70]:
# Альтернативный способ через бутсрап
booted_diff = []
booted_pvalue = []
size = max(a.size, b.size)
for _ in tqdm(range(10_000)):
    a_s = st.bernoulli.rvs(p=prob_1, size=size)
    b_s = st.bernoulli.rvs(p=prob_2, size=size)
    booted_diff.append(np.mean(a_s - b_s))
    booted_pvalue.append(proportion.proportions_chisquare([a_s.sum(), b_s.sum()], [a_s.size, b_s.size])[1])

md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
p_value_ci = 2 * (1 - st.norm.cdf(np.abs(md_ci / std_ci)))

100%|██████████| 10000/10000 [00:10<00:00, 943.78it/s]


In [71]:
print(f"Mean difference: {np.round(md_ci, 5)}, p-value: {p_value_ci} and confidence interval is: [{np.round(left_ci, 5)}, {np.round(right_ci, 5)}]")

if p_value_ci < alpha and not min(left_ci, right_ci) < 0 < max(left_ci, right_ci):
    print(f"Sample # {1 if md_ci > 0 else 2} mean is greater")
else:
    print("No significant difference")

Mean difference: -0.01909, p-value: 0.017205585591909767 and confidence interval is: [-0.03462, -0.00342]
Sample # 2 mean is greater


In [74]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Histogram(x=booted_pvalue, name='booted_pvalue',),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=booted_diff, name='booted_diff',),
    row=2, col=1
)

fig.update_layout(height=800, width=1000, title_text="After bootstrap")
fig.show()

In [73]:
# Ошибка первого рода, которая говорит нам о том, что группы почти не различаются между собой!
np.mean(np.array(booted_pvalue) < 0.05) # FPR

0.6715